In [1]:
import numpy as np
import cv2
import os,time
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_utils
import ipywidgets.widgets as widgets
from image_fun import bgr8_to_jpeg

In [2]:
import os,socket,sys,time
import spidev as SPI
import xgoscreen.LCD_2inch as LCD_2inch
from PIL import Image,ImageDraw,ImageFont
import numpy as np
import mediapipe as mp
from numpy import linalg
from xgolib import XGO

g_dog = XGO(port='/dev/ttyAMA0',version="xgolite")

In [3]:
#屏幕清除
mydisplay = LCD_2inch.LCD_2inch()
mydisplay.clear()
splash = Image.new("RGB", (mydisplay.height, mydisplay.width ),"black")
mydisplay.ShowImage(splash)

In [4]:
# Init camera 
cap = cv2.VideoCapture(0)  # 定义摄像头对象，参数0表示第一个摄像头 Define the camera object, parameter 0 represents the first camera
cap.set(3, 320) # set Width
cap.set(4, 240) # set Height
cap.set(5, 30)  #设置帧率 Setting the frame rate


True

In [5]:
image_widget = widgets.Image(format='jpg', width=320, height=240)

In [6]:
# Init tf model

MODEL_NAME = 'ssdlite_mobilenet_v2_coco_2018_05_09' #fast
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb' 
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt') 
NUM_CLASSES = 90 
IMAGE_SIZE = (12, 8) 
fileAlreadyExists = os.path.isfile(PATH_TO_CKPT) 

if not fileAlreadyExists:
    print('Model does not exsist !')
    exit

In [7]:
# LOAD GRAPH
print('Loading...')
detection_graph = tf.Graph() 
with detection_graph.as_default(): 
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid: 
        serialized_graph = fid.read() 
        od_graph_def.ParseFromString(serialized_graph) 
        tf.import_graph_def(od_graph_def, name='')
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True) 
category_index = label_map_util.create_category_index(categories)
print('Finish Load Graph..')

Loading...
Finish Load Graph..


In [8]:
print(type(category_index))

<class 'dict'>


In [9]:
print("dict['Name']: ", category_index[1]['name'])

dict['Name']:  person


In [10]:
# Main
t_start = time.time()
fps = 0
display(image_widget)
with detection_graph.as_default():
    with tf.compat.v1.Session(graph=detection_graph) as sess:
        while True:
            ret, frame = cap.read()
            ##############
            image_np_expanded = np.expand_dims(frame, axis=0) 
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0') 
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0') 
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0') 
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0') 
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

#             print('Running detection..') 
            (boxes, scores, classes, num) = sess.run( 
                [detection_boxes, detection_scores, detection_classes, num_detections], 
                feed_dict={image_tensor: image_np_expanded}) 

#             print('Done.  Visualizing..')
            vis_utils.visualize_boxes_and_labels_on_image_array(
                    frame,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=8)

            for i in range(0, 10):
                if scores[0][i] >= 0.5:
                    print(category_index[int(classes[0][i])]['name'])
                    objtype_str=category_index[int(classes[0][i])]['name']
            ##############
            fps = fps + 1
            mfps = fps / (time.time() - t_start)
            cv2.putText(frame, "FPS:" + str(int(mfps)), (10,25), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
            image_widget.value = bgr8_to_jpeg(frame)

            #显示在小车的lcd屏幕上
            b,g,r = cv2.split(frame)
            img = cv2.merge((r,g,b))
            imgok = Image.fromarray(img)
            mydisplay.ShowImage(imgok)

            # k = cv2.waitKey(1) & 0xff
            # if k == 27:# press 'ESC' to quit
            #     cap.release()
            #     break

    

Image(value=b'', format='jpg', height='240', width='320')

2025-05-15 14:49:58.849112: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 8640000 exceeds 10% of free system memory.
2025-05-15 14:49:59.375164: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 8640000 exceeds 10% of free system memory.
2025-05-15 14:49:59.778133: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 8640000 exceeds 10% of free system memory.
2025-05-15 14:50:00.169222: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 8640000 exceeds 10% of free system memory.
2025-05-15 14:50:00.537323: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 8640000 exceeds 10% of free system memory.


person
person
person
person
person
person
person
person
person
person
person
truck
person
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
person
tv
person
tv
person
tv
person
tv
person
person
person
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
tv
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
tv
person
tv
tv
person
tv
person
person
person
t

KeyboardInterrupt: 

In [11]:
cap.release()

mydisplay.clear()
splash = Image.new("RGB", (mydisplay.height, mydisplay.width ),"black")
mydisplay.ShowImage(splash)

del g_dog

#最后需要释放掉摄像头的占用 Finally, you need to release the camera's occupancy